# Importing Dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import time
import math
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate the angle between hip, knee and ankle 

In [2]:
def calculate_angles(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180:
        angle = 360 - angle
    return angle

# Implementing the model

In [8]:
cap = cv2.VideoCapture('Knee Exercise Video.mp4')

# Rep counter variables
rep = 0
stage = None

# variable to avoid multiple rep counts in a single up stage condition only
flag = 0

# defining timer variables
start_time = 0
end_time = 0
timer = 8

# variable to calculate angle between hip, knee and ankle
angle = 0

# width, height and frame of output window
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object.The output is stored in 'output.mp4' file.
out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc('P','I','M','1'), fps, (width, height))

# Define the Kalman filter
kf = cv2.KalmanFilter(4, 2)
kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kf.processNoiseCov = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32) * 0.03

# Initialize the Kalman filter with the initial position
t = 0

with mp_pose.Pose( min_detection_confidence=0.90,min_tracking_confidence=0.90) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Changing frame colors from BGR to RGB
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of hip, knee and ankle 
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            if (t==0):
                kf.statePost = np.array([knee[0], knee[1], 0, 0], np.float32)
                t = 1

            # Use the Kalman filter to predict the next position of the knee
            kf.predict()

            # Use the current position of the knee as the measurement
            kf.correct(np.array([knee[0], knee[1]], np.float32))

            # Get the predicted position of the knee
            knee = (kf.statePost[0]), (kf.statePost[1])

            
            # Calculate angle of each frame
            angle = calculate_angles(hip,knee,ankle)

            # Visualize angle
            cv2.putText(image, str("{0:.3f}".format(angle)), tuple(np.multiply(knee, [width, height]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,100), 2, cv2.LINE_AA)

            # rep counter condition
            if angle>140:
                stage = 'down'
                timer = 8
                if (end_time -start_time) < 8:
                    cv2.putText(image, 'Keep your knee bent',(350,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 2, cv2.LINE_AA)

                flag = 0

            if angle<140 and stage=='down':
                stage = 'up'
                start_time = end_time = time.time()

            elif angle<140 and stage=='up':
                end_time = time.time()

                if (end_time -start_time) >= 8 and flag == 0:
                    rep+=1
                    flag = 1

        except:
            pass
        
        # Render timer
        count = int(timer - (end_time -start_time))
        if (count<0):
            count = 0
        if (angle>140):
            count = timer
        cv2.circle(image,(780,40),42,(153,153,255),-1)
        cv2.putText(image,'Timer', (750,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2, cv2.LINE_AA)
        cv2.putText(image,str(count), (772,65), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)

        # Render rep counter
        cv2.rectangle(image,(0,0),(200,90),(153,153,255),-1)
        # Rep data
        cv2.putText(image,'Reps', (30,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (32,32,32), 2, cv2.LINE_AA)
        cv2.putText(image,str(rep), (45,60), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
        # Stage data
        cv2.putText(image,'Stage', (120,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2, cv2.LINE_AA)
        cv2.putText(image,stage, (125,60), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)

        # Make detection
        results = pose.process(image)

        # Changing frame colors from RGB to BGR
        image.flags.writeable = False
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # Render Detection
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

        # Write the frame into the file 'output.mp4'
        out.write(image)
        
        # Displaying the video
        cv2.imshow('Mediapipe feed',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    # Release video writer
    out.release()